In [1]:
from bs4 import BeautifulSoup
import requests
from difflib import SequenceMatcher

In [9]:
counter = 0
bad_film_names = []


def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

def parse_film_imdb(name: str):
    normal_name = name
    name = name.strip().split()
    name = '+'.join(name)
    
    url = f'https://www.imdb.com/find/?q={name}&ref_=nv_sr_sm'
    hdr = {'User-Agent': 'Mozilla/5.0'}
    page = requests.get(url, headers=hdr)
    soup = BeautifulSoup(page.text, 'html.parser')

    print(page)

    divs = soup.findAll('div', class_='ipc-metadata-list-summary-item__tc')
    
    good_div = []

    for div in divs:
        s = BeautifulSoup(str(div), 'html.parser')
        if f'>{normal_name}<'.lower() in str(s).lower() or similar(normal_name, s.find('a', attrs={'class': 'ipc-metadata-list-summary-item__t'}).text) >= 0.8:
            good_div.append(div)

    if good_div == []:
        global bad_film_names
        global counter
        counter += 1
        bad_film_names.append(normal_name)
        return normal_name, '', '', '', '', 'imdb'
    
    try:
        good_div = str(good_div[0])
    except:
        good_div = str(good_div)

    div_soup = BeautifulSoup(good_div, 'html.parser')

    url = 'https://imdb.com'+div_soup.find('a', class_='ipc-metadata-list-summary-item__t')['href']

    page = requests.get(url, headers=hdr)
    soup = BeautifulSoup(page.text, 'html.parser')

    try:
        score = soup.find('span', class_='sc-bde20123-1 iZlgcd').text
    except:
        score = ''

    ul = soup.find('ul', attrs={'class': 'ipc-inline-list ipc-inline-list--show-dividers sc-afe43def-4 kdXikI baseAlt'})
    ul = str(ul)
    if ul == '':
        print(url)
    ul_soup = BeautifulSoup(ul, 'html.parser')
    if len(ul_soup.find_all('li')) == 2:
        year = ul_soup.find('a', class_='ipc-link ipc-link--baseAlt ipc-link--inherit-color').text
        duration = ul_soup.find_all('li')[1].text
        age = ''
        return normal_name, year, age, duration, score, 'imdb'

    try:
        year = ul_soup.find('a', class_='ipc-link ipc-link--baseAlt ipc-link--inherit-color').text
    except:
        year = ''

    try:
        age = ul_soup.find_all('a', class_='ipc-link ipc-link--baseAlt ipc-link--inherit-color')[1].text
    except:
        age = ''

    try:
        duration = ul_soup.find_all('li')[2].text
    except:
        duration = ''

    return normal_name, year, age, duration, score, 'imdb'
    

parse_film_imdb('I Origins')

<Response [200]>


('I Origins', '2014', '12', '1h 46m', '7.3', 'imdb')